# Understanding the Data 

We will make use of the SMS Spam Collection Dataset provided by the UCI Machine Learning repository. According to the UCI site description, the SMS Spam Collection is a public set of SMS labeled messages that have been collected for mobile phone spam research. The data can be directly downloaded from [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).

In [1]:
import pandas as pd 

In [2]:
data = pd.read_table('SMSSpamCollection', delimiter = "\t",header=None, names=['label', 'sms_message'])
data.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Converting to Binary Labels

To make our ML model understand the targets, we need to convert them into 0/1 for binary classification.

In [3]:
# unique labels
print(data.iloc[:, 0].unique())

# function to convert string labels to binary
target_conversion = lambda x : 0 if x == 'ham' else 1
data['label'] = data.iloc[:, 0].apply(target_conversion)  

['ham' 'spam']


In [4]:
data.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# Number of training instances
data.shape

(5572, 2)

# Bag of Words 

Similar to our conversion of the target variable, we need to feed our model numeric inputs as features as well. 
Currently, our input feature is just the sms message itself. To convert this into numeric format, we will implement the Bag of Words (BoW) technique.

BoW essentially just counts the number of unique words in the entire vocabulary |V| and for each document, or sms in our case, creates a vector of counts of each of those vocabulary words.

We will first implement this algorithm from scratch, before making use of scikit-learn's Count Vectorizer method.

In [6]:
sample_documents = ['Hello, how are you!', 'Win money, win from home.', 'Call me now.', 'Hello, Call hello you tomorrow?']

##### Conversion to lower case

In [7]:
# First, we convert each word in the documents to lower case 
lower_case_documents = [word.lower() for word in sample_documents]

##### Remove punctuation

In [8]:
# remove all punctuation
import string 
sans_punctuation_documents = [t.translate(str.maketrans('', '', string.punctuation)) for t in lower_case_documents]
sans_punctuation_documents

['hello how are you',
 'win money win from home',
 'call me now',
 'hello call hello you tomorrow']

##### Word Tokenization

In [9]:
# Tokenizer - (word tokenizer to be specific), to split the entire corpus into individual words (based on a specific delimiter)
# In our custom implementation, we will consider words as strings separated by whitespace 
preprocessed_documents = [text.split(' ') for text in sans_punctuation_documents]
preprocessed_documents

[['hello', 'how', 'are', 'you'],
 ['win', 'money', 'win', 'from', 'home'],
 ['call', 'me', 'now'],
 ['hello', 'call', 'hello', 'you', 'tomorrow']]

##### Count Frequencies

Now that we have our data in the clean format we wanted, we can now create a Counter dictionary to count the frequency of unique words in each of the documents 
Here, we will create a Counter dictionary for each of the documents and store them in a `frequency_list`.

In [10]:
from collections import Counter 
from pprint import pprint
frequency_list = [Counter(text) for text in preprocessed_documents]
pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


##### Count Vectorizer
Having understood the basics of the BoW technique, we now make use of scikit-learn's own `Count Vectorizer` method, that will allow us to complete all of the above steps in a more concise manner.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

# First, let's look at how this built-in method performs on the sample documents above 
X = count_vector.fit_transform(sample_documents)
print(count_vector.get_feature_names_out())

['are' 'call' 'from' 'hello' 'home' 'how' 'me' 'money' 'now' 'tomorrow'
 'win' 'you']


Note that CountVectorizer takes care of the preprocessing/data cleaning steps that we had to implement ourselves for us. It makes use of certain parameter values to do this, including:

* `lowercase = True`
Default value True converts all text to lower case 

* `token_pattern = (?u)\\b\\w\\w+\\b`
The default regular expression ignores all punctuation marks, treating them as delimiters, and accepting strings of length >= 2 as individual tokens/words.

* `stop_words`
By default, this is set to None. However, we can set this to 'english' to use a built-in stop word list for English, to remove them from our corpus


In [12]:
# If we removed stop words as well
stop_words_vectorizer = CountVectorizer(stop_words='english')
stop_words_vectorizer.fit_transform(sample_documents)
print(stop_words_vectorizer.get_feature_names_out())
# Note that this is a much shorter list since the common (stop) words have been removed

['hello' 'home' 'money' 'tomorrow' 'win']


In [13]:
# View the implementation
# Based on the feature names, CountVectorizer converts our documents into arrays of number, with index i, j corresponding 
# to the count of word i in document j
# Word i may be found in the list generated from the get_feature_names_out() method as used above
doc_array = X.toarray()
doc_array

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]], dtype=int64)

In [14]:
# We can convert this matrix into a data frame for interpretation
frequency_matrix = pd.DataFrame(doc_array, columns=count_vector.get_feature_names_out())
frequency_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


One potential issue of using this technique is that when we have a large vocabulary size |V|, our feature_matrix
will be quite massive and sparse since most words will not exist in every document.

To mitigate this, we may make use of the stop words parameter, and also look at a technique called `tfidf` (to be implemented in another notebook)

# Train and Test sets 

Now, we look at implementing this BoW model on our actual dataset. But first, we will have to split our data 
into training and testing sets.

In [32]:
from sklearn.model_selection import train_test_split

# Splitting data into 80-20 Training and Test
X_train, X_test, y_train, y_test = train_test_split(data['sms_message'], data['label'], test_size=0.2, random_state=42)

print(f"Total number of rows: {data.shape[0]}")
print(f"Rows in training set: {X_train.shape[0]}")
print(f"Rows in test set: {X_test.shape[0]}")

Total number of rows: 5572
Rows in training set: 4457
Rows in test set: 1115


Now, we will apply the BoW processing to this dataset.

Important to note here that when we fit our CountVectorizer to the training dataset, it will create a feature matrix and learn the parameters based on the vocabulary in the training data. This means that if new words (not found in the training data) come up in the test data, they will not be considered by our model.

The following example shows this.

In [33]:
print(count_vector.fit_transform(sample_documents).toarray())
print(count_vector.get_feature_names_out())

# Counts of 12 features/words are learned

[[1 0 0 1 0 1 0 0 0 0 0 1]
 [0 0 1 0 1 0 0 1 0 0 2 0]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 1 0 2 0 0 0 0 0 1 0 1]]
['are' 'call' 'from' 'hello' 'home' 'how' 'me' 'money' 'now' 'tomorrow'
 'win' 'you']


In [34]:
# Only the 12 features/words learned above are used here
# Even though the test data contained 3 new words, they were simply ignored
test_doc = ['cricket football are great']
y = count_vector.transform(test_doc)
print(y.toarray())

[[1 0 0 0 0 0 0 0 0 0 0 0]]


In [37]:
# Instantiating the count vector
count_vector = CountVectorizer()

# fit and transform the training data
training_data = count_vector.fit_transform(X_train)

# only transform the test data (not fit)
test_data = count_vector.transform(X_test)


# Naive Bayes from scratch

Naive Bayes is a probabilistic classifier such that for a document `d`, out of all classes $c \in C$, it returns the class which has the maximum posterior probability given the document. This is denoted as:

$\hat{c} = \underset{c \in C}{\argmax}  P(c|d)$

To determine this class, the model makes use of the Bayes' rule such that:

$\hat{c} = \underset{c \in C}{\argmax}  P(c|d) = \underset{c \in C}{\argmax} \frac{P(d|c)*P(c)}{P(d)}$

However, since the denominator stays the same for each class, we can remove it and choose the class that maximises the simpler formula:

$\underset{c \in C}{\argmax} P(d|c)*P(c)$

The prior probability $P(c)$ is simply determined by the fraction of instances of class $c$ compared to all classes in $C$.

The likeilhood $P(d|c)$ is computed using a few simplifications.
First, we note that each documented may be represented simply through its features (or words in our case).

$P(d|c) = P(w_{1}, w_{2},...,w_{n}|c)$

Next, we make use of the (naive) conditional independence assumption that the probabilities $P(w_{i}|c)$ are independent given class $c$ and hence can be 'naively' multiplied as:

$P(w_{1}|c) * P(w_{2}|c) * ... * P(w_{n}|c)$

This results in our final equation (with i = position of each word i in the document):

$c_{NB} = \underset{c \in C}{\argmax} P(c) * \prod_{i \in positions} P(w_{i}|c)$

OR, more commonly used as:

$c_{NB} = \underset{c \in C}{\argmax} log(P(c)) + \sum_{i \in positions} log(P(w_{i}|c))$



##### Computing the prior probability

As mentioned above, the prior probability $P(c)$ is simply determined by the fraction of training instances with class $c$ as the target label, out of all labels.

Hence, $\hat{P}(c) = \frac{N_{c}}{N_{doc}}$ where $N_{c}$ is the number of documents of class $c$ in our training data and $N_{doc}$ is the total number of documents.

##### Computing the likelihood

We compute $P(w_{i}|c)$ as the fraction of times the word $w_{i}$ appears among all words in all documents of topic $c$. 

$\hat{P}(w_{i}|c) = \frac{w_{i}, c}{\sum_{w \in V}count(w, c)}$

##### Potential Issues

There are two potential problems we may run into with our approach.

1. A particular word that is in the training data, does not appear in a particular class. Meaning $\hat{P}(w_{i}|c)$ for that particular word and class would be 0, and it's logarithm will cause an error. To resolve this, we make use of add-one smoothing so that the count for such pairs is 1 instead of 0.

2. There may be words in the test data that were not encountered in the training set. To resolve this, we just ignore these new words and make sure to use only those that were found in the training set.

##### Example

We make use of the example shown in [Chapter 4 of Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/)

In [19]:
sample_train_data = {'label': [0,0,0,1,1], 'text': ['just plain boring', 'entirely predictable and lacks energy', 'no surprises and very few laughs',
'very powerful', 'the most fun film of the summer']}

sample_train_data = pd.DataFrame(sample_train_data)
sample_train_data

,label,text
0,0,just plain boring
1,0,entirely predictable and lacks energy
2,0,no surprises and very few laughs
3,1,very powerful
4,1,the most fun film of the summer


In [20]:
labels = sample_train_data.label.unique()

In [21]:
# Utility functions for our use
def unique_words(corpus):
    """ Given a body of text, returns a set of all unique words contained."""
    doc = [text.split(' ') for text in corpus]
    return set(word for text in doc for word in text)

def all_words(corpus):
    """ Given a body of text, returns all words contained"""
    doc = [text.split(' ') for text in corpus]
    return [word for text in doc for word in text] 

def num_class(c, classes=sample_train_data['label']):
    """ Given a class c, returns the number of documents of class c amongst all docs"""
    return (classes == c).sum() 

def count(w, c):
    """ Returns the occurences of w in bigdoc[c]"""
    i = big_doc[c].count(w)
    return i


In [22]:
# Create vocabulary V containing all unique words in training corpus
V = unique_words(sample_train_data['text'])
print(V)

{'of', 'few', 'the', 'entirely', 'boring', 'lacks', 'surprises', 'energy', 'very', 'plain', 'no', 'powerful', 'laughs', 'just', 'film', 'and', 'most', 'fun', 'predictable', 'summer'}


In [23]:
# To compute P(w|c), we need to find fraction of frequency of word w amongst all words in all documents of class c
# For this, we create a big document each, containing all text, corresponding to each class
doc_neg = sample_train_data.loc[sample_train_data['label'] == 0, 'text']
doc_pos = sample_train_data.loc[sample_train_data['label'] == 1, 'text']

doc_pos, doc_neg = all_words(doc_pos), all_words(doc_neg)
print(f"Vocabulary of Positive words: {doc_pos}")
print(f"Vocabulary of Negative words: {doc_neg}")

# Store in a generic big document:
big_doc = {0: doc_neg, 1: doc_pos}


Vocabulary of Positive words: ['very', 'powerful', 'the', 'most', 'fun', 'film', 'of', 'the', 'summer']
Vocabulary of Negative words: ['just', 'plain', 'boring', 'entirely', 'predictable', 'and', 'lacks', 'energy', 'no', 'surprises', 'and', 'very', 'few', 'laughs']


In [24]:
import numpy as np 
def train_naive_bayes(classes, d=sample_train_data['text']):
    
    log_prior_dict = dict()
    log_likelihood_dict = dict()
    for c in classes:
        n_doc = len(d) # number of total documents 
        n_c = num_class(c) # number of documents of class c 
        log_prior = np.log(n_c/n_doc)
        log_prior_dict[c] = log_prior
        word_likelihood = dict()
        for word in V:
            word_count = count(word, c) # occurences of word in bigdoc[c]
            log_likelihood = np.log((word_count + 1) / (len(big_doc[c]) + len(V)))
            word_likelihood[word] = log_likelihood
        log_likelihood_dict[c] = word_likelihood

    return log_prior_dict, log_likelihood_dict



In [25]:
log_prior, log_likelihood = train_naive_bayes(labels)
print(log_prior)

{0: -0.5108256237659907, 1: -0.916290731874155}


In [26]:
# We can now view some sample log probabilities and likelihoods through our implementation:
print(f"P(predictable|-) = {log_likelihood[0]['predictable']}, P(predictable|+) = {log_likelihood[1]['predictable']}")
print(f"P(no|-) = {log_likelihood[0]['no']}, P(no|+) = {log_likelihood[1]['no']}")
print(f"P(fun|-) = {log_likelihood[0]['fun']}, P(fun|+) = {log_likelihood[1]['fun']}")

# These values make sense since words that are more frequent in the positive documents, for example, have higher log likelihood


P(predictable|-) = -2.833213344056216, P(predictable|+) = -3.367295829986474
P(no|-) = -2.833213344056216, P(no|+) = -3.367295829986474
P(fun|-) = -3.5263605246161616, P(fun|+) = -2.6741486494265287


In [27]:
# Given a test example, we can now classify it based on our trained probabilities
def test_naive_bayes(test_doc, log_prior, log_likelihood, classes):

    """ Returns best class given a document"""
    class_probs = dict()
    for c in classes:
        class_probs[c] = log_prior[c]
        test_words = all_words(test_doc)
        for word in test_words:
            if word in V: # ignoring words not encountered in training corpus
                class_probs[c] += log_likelihood[c][word]
    return max(class_probs, key=class_probs.get) # return class with highest prob


In [28]:
test_doc_neg = ["predictable with no fun"]
test_doc_pos = ["it was a powerful movie with a lot of fun"]

preds_neg = test_naive_bayes(test_doc_neg, log_prior, log_likelihood, labels) 
preds_pos = test_naive_bayes(test_doc_pos, log_prior, log_likelihood, labels) 
print(f"Prediction of Negative Document: {preds_neg}")
print(f"Prediction of Positive Document: {preds_pos}")

Prediction of Negative Document: 0
Prediction of Positive Document: 1


# Implementing Naive Bayes using Scikit-Learn

In [47]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(training_data, y_train)

MultinomialNB()

In [50]:
predictions = nb.predict(test_data)
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# Model Evaluation

In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f"Accuracy: {accuracy_score(y_test, predictions):.3f}")
print(f"Precision: {precision_score(y_test, predictions):.3f}")
print(f"Recall: {recall_score(y_test, predictions):.3f}")
print(f"F1 score: {f1_score(y_test, predictions):.3f}")

Accuracy: 0.992
Precision: 1.000
Recall: 0.940
F1 score: 0.969
